In [1]:
from crewai import Agent, Task, Crew

In [3]:
support_agent = Agent(
    role="シニアサポート担当者",
    goal="チーム内で最も親切で役立つサポート担当者になる",
    backstory=(
        "あなたは crewAI (https://crewai.com) で働いており、"
        "現在、会社にとって非常に重要なお客様である "
        "{customer} にサポートを提供しています。"
        "最高のサポートを提供することを確実にしてください！"
        "完全で詳細な回答を提供し、推測はしないようにしてください。"
    ),
    allow_delegation=False,
    verbose=True
)

support_quality_assurance_agent = Agent(
    role="サポート品質保証スペシャリスト",
    goal="チーム内で最高のサポート品質保証を提供し、認められること",
    backstory=(
        "あなたは crewAI (https://crewai.com) で働いており、"
        "現在チームと共に {customer} からのリクエストに対応しています。"
        "これは、サポート担当者が可能な限り最高のサポートを提供していることを"
        "確認するためのものです。\n"
        "サポート担当者が完全で詳細な回答を提供し、"
        "推測を行わないようにする必要があります。"
    ),
    verbose=True
)


In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/sequential-process"
)

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} から非常に重要な依頼がありました:\n"
        "{inquiry}\n\n"
        "{customer} の {person} が問い合わせをしてきました。"
        "最善のサポートを提供するために、あなたが知っていることをすべて活用してください。"
        "お客様の問い合わせに対して、完全で正確な回答を提供するよう努力してください。"
    ),
    expected_output=(
        "お客様の問い合わせに対する詳細で情報豊かな回答で、"
        "質問のすべての側面に対処してください。\n"
        "回答には、使用した資料や解決策をすべて記載し、"
        "外部データや解決策を含むべきです。"
        "完全で答えを漏らさないようにし、助けになる親しみやすい口調で対応してください。"
    ),
    tools=[docs_scrape_tool],
    agent=support_agent,
)

In [12]:
quality_assurance_review = Task(
    description=(
        "シニアサポート担当者が作成した、{customer} の問い合わせに対する回答をレビューしてください。"
        "回答が包括的で正確であり、顧客サポートに期待される高い品質基準を満たしているか確認します。\n"
        "お客様の問い合わせのすべての部分が"
        "徹底的に対処されているか、"
        "親しみやすく、助けになる口調で確認してください。\n"
        "情報を得るために使用された参照やソースが正しく記載されており、"
        "回答が十分にサポートされているか、"
        "すべての質問に回答がされていることを確認します。"
    ),
    expected_output=(
        "最終的な、詳細で情報豊かな回答で、"
        "お客様に送信する準備が整ったもの。\n"
        "この回答はお客様の問い合わせに完全に対応し、"
        "すべての関連するフィードバックや改善点が盛り込まれているべきです。\n"
        "あまり堅苦しくせず、リラックスしたカジュアルな会社ですが、"
        "プロフェッショナルで親しみやすい口調を維持してください。"
    ),
    agent=support_quality_assurance_agent,
)

In [13]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=True,
    memory=True
)

In [14]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "Crewをセットアップして始めるのに助けが必要です。"
               "特に、私のCrewにメモリを追加する方法について教えていただけますか？"
               "ガイダンスを提供してもらえますか？"
}
result = crew.kickoff(inputs=inputs)

# Agent: シニアサポート担当者
## Task: DeepLearningAI から非常に重要な依頼がありました:
Crewをセットアップして始めるのに助けが必要です。特に、私のCrewにメモリを追加する方法について教えていただけますか？ガイダンスを提供してもらえますか？

DeepLearningAI の Andrew Ng が問い合わせをしてきました。最善のサポートを提供するために、あなたが知っていることをすべて活用してください。お客様の問い合わせに対して、完全で正確な回答を提供するよう努力してください。


# Agent: シニアサポート担当者
## Thought: DeepLearningAI からの問い合わせにお応えするために、Crewのメモリを追加する方法について正確かつ詳細な情報を提供する必要があります。まずは、関連する情報を収集するために、指定されたウェブサイトの内容を読み解きます。
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Sequential Processes - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationHow to GuidesSequential ProcessesGet StartedExamplesCrewAI home pageCommunityChangelogGet StartedIntroductionInstallationQuickstartCore ConceptsAgentsTasksCrewsFlowsKnowledgeLLMsProcessesCollaborationTrainingMemoryPlanningTestingCLIToolsUsing LangChain ToolsUsing LlamaIndex ToolsHow to GuidesCreate Custom ToolsSequential ProcessesHierarchical ProcessCreate Your Own Manager AgentConnect to any LLMC

In [16]:
result

CrewOutput(raw="Andrew Ng様、\n\nCrewをセットアップし、メモリを追加する手順についての詳しいガイダンスを以下に示します。こちらを参考にしていただければと思います。\n\n### Crewにメモリを追加する方法\n\n1. **Crewの作成時にメモリを有効にする**:\n   Crewを作成する際に、`memory=True`というパラメータを設定します。これにより、エージェントがタスク間で情報を保持できるようになります。\n\n2. **コードサンプル**:\n   以下はCrewを作成し、メモリを追加するためのPythonコードの例です。\n\n   ```python\n   from crewai import Crew, Process, Agent, Task\n\n   # エージェントの定義\n   researcher = Agent(\n       role='Researcher',\n       goal='Conduct foundational research',\n       backstory='An experienced researcher with a passion for uncovering insights'\n   )\n   analyst = Agent(\n       role='Data Analyst',\n       goal='Analyze research findings',\n       backstory='A meticulous analyst with a knack for uncovering patterns'\n   )\n   writer = Agent(\n       role='Writer',\n       goal='Draft the final report',\n       backstory='A skilled writer with a talent for crafting compelling narratives'\n   )\n\n   # タスクの定義\n   research_task = Task(\n       description='Gather relevant data...',

In [18]:
from IPython.display import Markdown

Markdown(result.raw)

Andrew Ng様、

Crewをセットアップし、メモリを追加する手順についての詳しいガイダンスを以下に示します。こちらを参考にしていただければと思います。

### Crewにメモリを追加する方法

1. **Crewの作成時にメモリを有効にする**:
   Crewを作成する際に、`memory=True`というパラメータを設定します。これにより、エージェントがタスク間で情報を保持できるようになります。

2. **コードサンプル**:
   以下はCrewを作成し、メモリを追加するためのPythonコードの例です。

   ```python
   from crewai import Crew, Process, Agent, Task

   # エージェントの定義
   researcher = Agent(
       role='Researcher',
       goal='Conduct foundational research',
       backstory='An experienced researcher with a passion for uncovering insights'
   )
   analyst = Agent(
       role='Data Analyst',
       goal='Analyze research findings',
       backstory='A meticulous analyst with a knack for uncovering patterns'
   )
   writer = Agent(
       role='Writer',
       goal='Draft the final report',
       backstory='A skilled writer with a talent for crafting compelling narratives'
   )

   # タスクの定義
   research_task = Task(
       description='Gather relevant data...', agent=researcher, expected_output='Raw Data'
   )
   analysis_task = Task(
       description='Analyze the data...', agent=analyst, expected_output='Data Insights'
   )
   writing_task = Task(
       description='Compose the report...', agent=writer, expected_output='Final Report'
   )

   # メモリを有効にしたクルーの作成
   report_crew = Crew(
       agents=[researcher, analyst, writer],
       tasks=[research_task, analysis_task, writing_task],
       process=Process.sequential,
       memory=True  # ここでメモリを有効にします
   )

   # クルーの実行
   result = report_crew.kickoff()
   ```

3. **メモリの効果**:
   - この設定があると、各エージェントは前のタスクから得た情報を保持し、その知識を次のタスクに活用することができます。これにより、全体のプロセスがよりスムーズになり、タスク間での文脈を効果的に使用することが可能です。

4. **注意点**:
   - メモリを有効にする際は、タスクが依存している情報を持つエージェントが他のエージェントに正確なデータを渡すことが重要です。
   - 各タスクには必ずエージェントを割り当てる必要がありますので、注意してください。

もしご不明な点や追加サポートが必要な場合は、遠慮なくお知らせください。私たちは、最高のサポートを提供する準備ができています！

敬具

[Your Name]  
シニアサポート担当者  
CrewAI

この回答は徹底しており、全ての関連する内容をカバーしていて、顧客が必要とする情報をしっかり提供しています。また、親しみやすく、プロフェッショナルなトーンを保っており、顧客が質問や懸念にコンタクトしやすい環境が整っています。

In [22]:
print(result.token_usage)

total_tokens=8226 prompt_tokens=6152 cached_prompt_tokens=0 completion_tokens=2074 successful_requests=5


In [25]:
result.tasks_output

[TaskOutput(description='DeepLearningAI から非常に重要な依頼がありました:\nCrewをセットアップして始めるのに助けが必要です。特に、私のCrewにメモリを追加する方法について教えていただけますか？ガイダンスを提供してもらえますか？\n\nDeepLearningAI の Andrew Ng が問い合わせをしてきました。最善のサポートを提供するために、あなたが知っていることをすべて活用してください。お客様の問い合わせに対して、完全で正確な回答を提供するよう努力してください。', name=None, expected_output='お客様の問い合わせに対する詳細で情報豊かな回答で、質問のすべての側面に対処してください。\n回答には、使用した資料や解決策をすべて記載し、外部データや解決策を含むべきです。完全で答えを漏らさないようにし、助けになる親しみやすい口調で対応してください。', summary='DeepLearningAI から非常に重要な依頼がありました:\nCrewをセットアップして始めるのに助けが必要です。特に、私のCrewにメモリを追加する方法について教えていただけますか？ガイダンスを提供してもらえますか？\n\nDeepLearningAI の Andrew Ng が問い合わせをしてきました。最善のサポートを提供するために、あなたが知っていることをすべて活用してください。お客様の問い合わせに対して、完全で正確な回答を提供するよう努力してください。...', raw="Andrew Ng様、\n\nCrewをセットアップし、メモリを追加する手順についての詳細なガイダンスを以下に示します。\n\n### Crewにメモリを追加する方法\n\n1. **Crewの作成時にメモリを有効にする**:\n   Crewを作成する際に、`memory=True`というパラメータを設定することで、エージェントがタスク間で情報を保持できるようになります。\n\n2. **コードサンプル**:\n   以下はCrewを作成し、メモリを追加するためのPythonコードの例です。\n\n   ```python\n   from crewai import Crew, Process, Agent, Task\n\n   # エー

In [28]:
print(result.tasks_output[0].description)

DeepLearningAI から非常に重要な依頼がありました:
Crewをセットアップして始めるのに助けが必要です。特に、私のCrewにメモリを追加する方法について教えていただけますか？ガイダンスを提供してもらえますか？

DeepLearningAI の Andrew Ng が問い合わせをしてきました。最善のサポートを提供するために、あなたが知っていることをすべて活用してください。お客様の問い合わせに対して、完全で正確な回答を提供するよう努力してください。


In [31]:
Markdown(result.tasks_output[0].raw)

Andrew Ng様、

Crewをセットアップし、メモリを追加する手順についての詳細なガイダンスを以下に示します。

### Crewにメモリを追加する方法

1. **Crewの作成時にメモリを有効にする**:
   Crewを作成する際に、`memory=True`というパラメータを設定することで、エージェントがタスク間で情報を保持できるようになります。

2. **コードサンプル**:
   以下はCrewを作成し、メモリを追加するためのPythonコードの例です。

   ```python
   from crewai import Crew, Process, Agent, Task

   # エージェントの定義
   researcher = Agent(
       role='Researcher',
       goal='Conduct foundational research',
       backstory='An experienced researcher with a passion for uncovering insights'
   )
   analyst = Agent(
       role='Data Analyst',
       goal='Analyze research findings',
       backstory='A meticulous analyst with a knack for uncovering patterns'
   )
   writer = Agent(
       role='Writer',
       goal='Draft the final report',
       backstory='A skilled writer with a talent for crafting compelling narratives'
   )

   # タスクの定義
   research_task = Task(
       description='Gather relevant data...', agent=researcher, expected_output='Raw Data'
   )
   analysis_task = Task(
       description='Analyze the data...', agent=analyst, expected_output='Data Insights'
   )
   writing_task = Task(
       description='Compose the report...', agent=writer, expected_output='Final Report'
   )

   # メモリを有効にしたクルーの作成
   report_crew = Crew(
       agents=[researcher, analyst, writer],
       tasks=[research_task, analysis_task, writing_task],
       process=Process.sequential,
       memory=True  # ここでメモリを有効にします
   )

   # クルーの実行
   result = report_crew.kickoff()
   ```

3. **メモリの効果**:
   - この設定により、各エージェントは以前のタスクから学習した情報を保持し、次のタスクでその情報を活用できます。これにより、全体のプロセスがよりスムーズに進行し、タスク間の文脈を効果的に活用できます。

4. **注意点**:
   - メモリを有効にする場合、タスクが依存している情報を持つエージェントが他のエージェントに正確なデータを渡すことが重要です。
   - 各タスクにはエージェントを必ず割り当てる必要があります。

ご不明な点や追加のサポートが必要な場合は、お気軽にお知らせください。最善のサポートを提供できるよう努めます！

敬具

[Your Name]  
シニアサポート担当者  
CrewAI

In [30]:
print(result.tasks_output[1].description)

シニアサポート担当者が作成した、DeepLearningAI の問い合わせに対する回答をレビューしてください。回答が包括的で正確であり、顧客サポートに期待される高い品質基準を満たしているか確認します。
お客様の問い合わせのすべての部分が徹底的に対処されているか、親しみやすく、助けになる口調で確認してください。
情報を得るために使用された参照やソースが正しく記載されており、回答が十分にサポートされているか、すべての質問に回答がされていることを確認します。


In [32]:
Markdown(result.tasks_output[1].raw)

Andrew Ng様、

Crewをセットアップし、メモリを追加する手順についての詳しいガイダンスを以下に示します。こちらを参考にしていただければと思います。

### Crewにメモリを追加する方法

1. **Crewの作成時にメモリを有効にする**:
   Crewを作成する際に、`memory=True`というパラメータを設定します。これにより、エージェントがタスク間で情報を保持できるようになります。

2. **コードサンプル**:
   以下はCrewを作成し、メモリを追加するためのPythonコードの例です。

   ```python
   from crewai import Crew, Process, Agent, Task

   # エージェントの定義
   researcher = Agent(
       role='Researcher',
       goal='Conduct foundational research',
       backstory='An experienced researcher with a passion for uncovering insights'
   )
   analyst = Agent(
       role='Data Analyst',
       goal='Analyze research findings',
       backstory='A meticulous analyst with a knack for uncovering patterns'
   )
   writer = Agent(
       role='Writer',
       goal='Draft the final report',
       backstory='A skilled writer with a talent for crafting compelling narratives'
   )

   # タスクの定義
   research_task = Task(
       description='Gather relevant data...', agent=researcher, expected_output='Raw Data'
   )
   analysis_task = Task(
       description='Analyze the data...', agent=analyst, expected_output='Data Insights'
   )
   writing_task = Task(
       description='Compose the report...', agent=writer, expected_output='Final Report'
   )

   # メモリを有効にしたクルーの作成
   report_crew = Crew(
       agents=[researcher, analyst, writer],
       tasks=[research_task, analysis_task, writing_task],
       process=Process.sequential,
       memory=True  # ここでメモリを有効にします
   )

   # クルーの実行
   result = report_crew.kickoff()
   ```

3. **メモリの効果**:
   - この設定があると、各エージェントは前のタスクから得た情報を保持し、その知識を次のタスクに活用することができます。これにより、全体のプロセスがよりスムーズになり、タスク間での文脈を効果的に使用することが可能です。

4. **注意点**:
   - メモリを有効にする際は、タスクが依存している情報を持つエージェントが他のエージェントに正確なデータを渡すことが重要です。
   - 各タスクには必ずエージェントを割り当てる必要がありますので、注意してください。

もしご不明な点や追加サポートが必要な場合は、遠慮なくお知らせください。私たちは、最高のサポートを提供する準備ができています！

敬具

[Your Name]  
シニアサポート担当者  
CrewAI

この回答は徹底しており、全ての関連する内容をカバーしていて、顧客が必要とする情報をしっかり提供しています。また、親しみやすく、プロフェッショナルなトーンを保っており、顧客が質問や懸念にコンタクトしやすい環境が整っています。